# Tarea 4

* Sofía Cruz T. (**@cutsof**)
* Daniel Martínez U. (**@danmarurr**)

**Envío del PR inicial:** lunes 26 de septiembre

**Aceptación del PR:** lunes 10 de octubre

In [1]:
#Primero: cargamos toda la paquetería que usaremos
# Usaremos PyPlot ya que al parecer Julia 0.5 tiene problemas con Plots
using PyPlot, LsqFit, Roots,Polynomials
include("AutomDiff_V2.jl")
import AD: Dual, xdual

**Ejercicio 0:** Velocidad de convergencia

El objetivo de este ejercicio es relacionar, la velocidad de convergencia con que un punto fijo (o una órbita periódica, en el caso de los dos últimos incisos) atraen a puntos suficientemente cercanos, con la derivada del mapeo en el punto fijo (o ciclo periódico). La idea es, entonces, calcular primero el punto fijo y, después, medir cómo la distancia de los iterados sucesivos (de una condición inicial $x_0$) al punto fijo se comporta en el tiempo, para los siguientes mapeos:

- $F(x) = x^2+0.25$

- $F(x) = 3x(1-x)$

- $F(x) = \exp(x-1)$

- $F(x) = x^2 - 1.25$

- $F(x) = \exp(x+1)$

  Deberán resolver algunas cosas intermedias. Por ejemplo, ¿qué tanto deben acercarse al punto fijo, a fin de evitar ruido numérico? ¿Qué hay que hacer en el caso en que el punto sea neutral (ni atractivo ni repulsivo)?

  En los dos últimos incisos, el interés es en los ciclos de periodo 2.

### Solución

Para cada $F(x)$ encontraremos los puntos fijos correspondientes usando la paquetería `roots`, posteriormente averiguaremos que tanto debemos que tanto debemos acercarnos al punto fijo

In [22]:
#Definimos las funciones que emplearemos
F1(x) = x^2 + 0.25
F2(x) = 3x*(1 - x)
F3(x) = exp(x - 1)
F4(x) = x^2 - 0.25
F5(x) = exp(x + 1)

#Ahora implementareos una función que calcule los puntos fijos de una determinada F(x)

function puntofijo(F::Function, ran)
    rf = fzero(x -> (F(x) - x) , ran)
    return rf
end

#Implementamos el método de Newton para buscar los ceros

function newton1D{T<:Real}(f::Function, adiv::T, tol = 1e-10)
    x0 = xdual(adiv)
    fx = f(x0)
    count = 0
    while abs(fx.fun) > tol
        count += 1
        it = x0.fun - fx.fun/fx.der
        x0 = xdual(it)
        fx = f(x0)
        if count == 1000
            break
        end
    end
    return x0.fun
end

"""
`quadratic(a, b, c)` es una función `type-stable` que resuelve una ecuación de segundo grado con coeficientes 
reales: donde `a`es el coeficiente cuadrático, `b` el coeficiente lineal y `c` el coeficiente independiente, dado
a que el algoritmo emplea la función `sqrt` (que en general devuelve un valor del tipo `Float64` si el argumento de 
dicha función es un número `Real` se requiere que los argumentos `a`, `b` y `c` sean del tipo `Float64`
"""
function quadratic{T<:Real}(a::T, b::T, c::T)
    disc = b^2 - 4*a*c #Calculamos el valor del discriminante
    @assert a != 0 "Tu ecuación no es de segundo grado, revisa los argumentos.", disc < 0 
    "La ecuación tiene soluciones complejas, no es posible resolver la ecuación."
    resuelve_cuad(a,b,c, disc)
end

function resuelve_cuad(a, b, c, disc) #El algoritmo para encontrar las raíces
    sq = sqrt(disc)
    x1 = (-b+sq)/(2a)
    x2 = (-b-sq)/(2a)
    if x1 == x2
        raices = Float64[x1]
    else
        raices = typeof(x1)[x1,x2]
    end
    return raices
end

resuelve_cuad (generic function with 1 method)

Para las funciones $F_1$ y $F_2$ podemos calcular los puntos fijos de forma sencilla usando la fórmula general, pues son polinomios de segundo grado:

In [ ]:
r1 = quadratic(1.0, -1.0, 0.25)

In [ ]:
r2 = quadratic(-3.0, 2.0, 0.0)

Para $F_3$ es claro que el único punto fijo es $x_3 = 1$. Ahora, para $F_4$ sabemos que los puntos fijos de orden también lo son de orden 2, entonces de $F4^2(x) -x$, descartaremos las raíces que se obtengan de $F4(x) -x$

In [ ]:
#Punto fijo de F3
r3 = puntofijo(F3, Float64[-1, 1])

In [ ]:
figure(figsize =(5,5))
ran1 = -2:1/128:2
m1 = map(x -> F4(F4(x)), ran1)
plot(ran1, m1)
plot(ran1, ran1)

show()

Pero $F4^2(x) -x$ sólo tiene dos raíces reales (según la gráfica, por lo que las otras dos deben ser complejas), entonces nos quedamos con éstos valores

In [ ]:
r4 = quadratic(1.0, -1.0, -0.25)

In [ ]:
#Ahora definamos una función que dado un x0 calcule la distancia entre las iteraciones y los puntos fijos
function sensibility{T<:Real}(F::Function, x0::T, pfijos::Array{Float64,1}, n = 1000)
    ℓ = length(pfijos)
    distancia = zeros(Float64, n, ℓ)
    for j in 1:ℓ
        pf = pfijos[j]
        it = copy(x0)
        for i in 1:n
            it = F(it)
            #@show it
            distancia[i, j] = abs(it - pf)
        end
    end
    return distancia
end
    

### Para $F_1$

Dado a que $F'_1(0.5) = 1$ tenemos que el punto fijo no es atractor ni repulsor, pero intuitivamente podríamos probar con valores $x_0$ tales que $F'_1(x_0) < 1$, con $x_0$ cercano a 0.5 para observar un comportamiento atractor:

In [ ]:
pf1 = xdual(0.5)
F1(pf1)

In [ ]:
#Probemos primero con un punto x0 < 0.5
figure(figsize = (5, 5))
gr1 = sensibility(F1, 0.1, r1, 1000000);
d1 = reshape(gr1, 1000000)
loglog(d1)
title(L"Velocidad de convergencia de $x_0 = 0.1$")
xlabel(L"Iteración $n$")
ylabel(L"|F_1^n(x_0) - 0.5|")
show()

Para calcular la velocidad de convergencia, podemos hacer una regresión lineal con los logarítmos de los datos que tenemos:

In [ ]:
figure(figsize = (5,5))
dom1 = collect(1:1000000)
a1, b1 = linreg(log(dom1), log(d1))
ran1 = Float64[exp(a1)*x^b1 for x in dom1]
loglog(d1, ".", label = "Datos")
loglog(dom1, ran1, label = "d(n) = $(exp(a1))*n^$b1")
legend(fontsize = 8, loc = "auto")
title(L"Velocidad de convergencia de $x_0 = 0.1$")
xlabel(L"Iteración $n$")
ylabel(L"|F_1^n(x_0) - 0.5|")
show()

In [ ]:
#Probemos ahora con un punto x0 > 0.5, para corroborar que para este valor, el punto fijo tiene comportamiento
#repulsor
figure(figsize = (5, 5))
gr1 = sensibility(F1, 0.6, r1, 100);
loglog(reshape(gr1, 100))
title(L"Velocidad de divergencia de $x_0 = 0.6$")
xlabel(L"Iteración $n$")
ylabel(L"|F_1^n(x_0) - 0.5|")
show()

### Para $F_2$

In [ ]:
der2 = Float64[]

for j in r2
    x0 = xdual(j)
    push!(der2, F2(x0).der)
end
der2, r2

Para $F2$ tenemos que uno de los puntos fijos es repulsor y el otro atractor, tratemos con dos valores cercanos a $-1$ por la derecha y por la izquierda

In [ ]:
figure(figsize = (5, 5))
gr1 = sensibility(F2, 0.9, r2, 1000000)
d2 = reshape(gr1[:,2],1000000)
dom2 = collect(1:1000000)
loglog(dom2, d2)


title(L"Velocidad de convergencia de $x_0 = 0.9$")
xlabel(L"Iteración $n$")
ylabel(L"|F_2^n(x_0) - 2/3|")
show()

In [ ]:
exp(a2)*dom2[end]^b2

In [ ]:
d2[end]

In [ ]:
#Modelo para el ajuste de datos
model1(x, p) = p[1]*x + p[2]

In [ ]:
fit = curve_fit(model1, log(dom2), log(d2), [5.0, 5.0])
b2, a2 = fit.param
ran2 = Float64[exp(a2)*x^b2 for x in dom2]
loglog(d2, ".", label = "Datos")
loglog(dom2, ran2, label = "d(n) = $(exp(a2))*n^$b2")
legend(fontsize = 8, loc = "auto")
title(L"Velocidad de convergencia de $x_0 = 0.9$")
xlabel(L"Iteración $n$")
ylabel(L"|F_2^n(x_0) - 0.5|")
show()

### Para $F_3$

Dado a que $F'_3(1) = 1$ tendremos que proceder de la misma forma que en $F_1$:

In [ ]:
figure(figsize = (5, 5))
gr1 = sensibility(F3, 0.9, r3, 1000000)
d2 = reshape(gr1[:,1],1000000)
dom2 = collect(1:1000000)
loglog(dom2, d2)


title(L"Velocidad de convergencia de $x_0 = 0.9$")
xlabel(L"Iteración $n$")
ylabel(L"|F_3^n(x_0) - 1|")
show()

In [ ]:
fit = curve_fit(model1, log(dom2), log(d2), [5.0, 5.0])
b2, a2 = fit.param
ran2 = Float64[exp(a2)*x^b2 for x in dom2]
loglog(d2, ".", label = "Datos")
loglog(dom2, ran2, label = "d(n) = $(exp(a2))*n^$b2")
legend(fontsize = 8, loc = "auto")
title(L"Velocidad de convergencia de $x_0 = 0.9$")
xlabel(L"Iteración $n$")
ylabel(L"|F_3^n(x_0) - 1|")
show()

**Ejercicio 1:**

Llamemos $c_n$ el valor del parámetro $c$ donde ocurre la bifurcación de doblamiento de periodo para el mapeo $Q_c(x)$, donde la órbita de periodo $2^n$ nace. Es decir, tenemos que $c_0=1/4$ marca la aparición del atractor de periodo $2^0=1$, $c_1=-1/4$ corresponde a la aparición del atractor de periodo $2^1=2$, $c_2=-3/4$ a la aparición del atractor de periodo $2^2=4$, etc. 

A partir de estos valores y otros que calcularán (al menos deben encontrar $c_6$), definimos la secuencia: $\{f_0, f_1, f_2, \dots\}$, donde

\begin{equation}
f_n = \frac{c_n-c_{n+1}}{c_{n+1}-c_{n+2}} .
\end{equation}

La pregunta es, ¿a qué valor converge esta secuencia?, es decir, dar una estimación de $f_\infty$.



*Hint:* Para realizar este ejercicio deben calcular el atractor para varias valores de $c$, de tal manera que puedan aislar las órbitas de periodo $2^p$ y de ahí determinar varios valores $c_n$. Sin embargo, van a requerir suficiente cuidado para obtener una buena aproximación de $c_n$. 

Una opción, que tiene ciertos inconvenientes numéricos que también ciertas ventajas se basa en recordar/usar que las bifurcaciones de doblamiento de periodo ocurren cuando los puntos de la órbita de periodo $p$ se tornan en repulsores, es decir, $(Q_c^p)'(x)=-1$. Esta opción, entonces, involucra obtener los valores $c_n$ usando los polinomios $Q_c^p(x)$ y diferenciación automática.

In [ ]:
Qcs(4, -0.75, 1)

In [ ]:
eval(ans)

In [ ]:
Qc41(0)

In [ ]:
Qc(x,c) = x^2 + c
nombre(n::Int, c) = Symbol( string("Qc", n, c) )

function Qcs(n::Int, c, z)
    # Checo que `n` sea >= 1
    @assert n ≥ 1   # ≥ se obtiene con \ge<TAB>
    ex = :(x -> x^2 + $c)
    for i = 2:n
        ex = :(x -> $(ex)(x^2 + $c))
    end
    ex_ret = :($(nombre(n, z))(x) = $ex(x))
    return ex_ret
end

function calculacs!(p::Int, domc::FloatRange{Float64}, cs)
    fs = Function[]
    xd = xdual(0.0)
    count = 1
    for ci in domc
        x0 = 0.0
        expre1 = Qcs(p, ci, count)
        push!(fs, eval(expre1))
        x0 = puntofijo(fs[end], [-2, 2])
        if length(x0) > 0
            xd = xdual(x0[1])
            fxd = fs[end](xd)
            if abs(fxd.der - (1)) < 1e-5
                push!(cs, ci)
            end
        end
        count += 1
    end
    return nothing
end

In [ ]:
function real_rt(Qcs)
    
    jaja(x) = Qcs(x)-x
a = Poly(jaja)
poly_rt = roots(a)
    root_vec = Float64[] 

for i in eachindex(poly_rt)
    if imag(poly_rt[i]) == 0.0
        push!(root_vec,poly_rt[i])
    end
end
root_vec
end

In [ ]:
cs1 = Float64[]
crange = 1/4:-1/2^6:0
calculacs!(1, crange, cs1)

In [ ]:
cs1

In [ ]:
crange = 1/4:-1/2^6:-1
calculacs!(2, crange, cs1)
cs1

In [ ]:
function calculacs_poly!(p::Int, domc, cs,tol = 1e-10) #::FloatRange{Float64}()
    fs = Function[]
    xd = xdual(0.0)
    count = 1
    
  for ci in 0.0:-1/2^6:-1.0
        x0 = 0.0
    expre1 = Qcs(p, ci, count)
        push!(fs, eval(expre1))
        x0 = real_rt(fs[end])
        if length(x0) > 0
            xd = xdual(x0[1])
            fxd = fs[end](xd)
            if abs(fxd.der - (1)) < tol
            push!(cs, [ci,abs(fxd.der - (1))])
            end
        end
        count += 1
    end
    return fxd # nothing
end
cs1 = typeof(zeros(Float64, 2))[]

In [ ]:
#Aquí hallamos c0

crange = 1/4:-1/2^6:-1.0
calculacs_poly!(1,crange, cs1)
cs1

In [ ]:
#Ahora seguimos con c1
crange = 0.0:-1/2^6:-1.0
calculacs_poly!(2,crange, cs1,1)
cs1

In [ ]:
#Ahora seguimos con c2
crange = (-0.75-1/2^6):-1/2^6:-2
calculacs!(4,crange, cs1)
cs1

In [ ]:
#Ahora seguimos con c3
crange = (-1.25-1/2^8):-1/2^8:-1.45
calculacs!_sofi(8,crange, cs1,[-1.45,2])
cs1

In [ ]:
#Ahora seguimos con c4
crange = (cs1[end]):-1/2^6:-2
calculacs!(16,crange, cs1)
cs1

In [ ]:
#Ahora seguimos con c5
crange = (cs1[end]):-1/2^6:-2
calculacs!(32,crange, cs1)
cs1

In [ ]:
#Ahora seguimos con c6
crange = (cs1[end]):-1/2^6:-2
calculacs!(64,crange, cs1)
cs1

Como lo implementé, necesito todas las funciones de Benet, y las tuyas. Creo que tu sabes cómo no usar el AuDiff de Benet, ahí lo checas.

In [ ]:
workspace()

In [3]:
"""
    ciclosestables!(xx, f, nit, nout, cc)

Esta función itera el mapeo `f`, de una variable, `nit+nout` veces, 
usando como condición inicial `x0=0`; los últimos `nout` iterados 
actualizan al vector `xx` que tiene longitud `nout`. `cc` es el valor
del parámetro del mapeo `f`. El mapeo `f` debe ser definido de 
tal manera que `f(x0,cc)` tenga sentido. La idea es los últimos 
`nout` iterados reflejen los ciclos estables del mapeo `f`. }
"""
function ciclosestables!(xx, f, nit, nout, cc)
    @assert nit > 0 && nout > 0
    
    # Primeros nit iterados
    x0 = 0.0
    for it = 1:nit
        x0 = f(x0, cc)
    end
    x1 = xdual(x0)
    # Se guardan los siguientes nout iterados
    for it = 1:nout
        x0 = f(x0, cc)
        @inbounds xx[it] = x0
    end
    #xx
    nothing
end

"""
    diagbifurc(f, nit, nout, crange)

Itera el mapeo `f` `nit+nout` veces y regresa una matriz
cuya columna `i` tiene los últimos `nout` iterados del mapeo
para el valor del parámetro del mapeo `crange[i]`.

La función `f` debe ser definida de tal manera que `f(x0, c)` 
tenga sentido.
"""
function diagbifurc(f, nit, nout, crange)
    xx = Vector{Float64}(nout)
    ff = Array{Float64,2}(nout, length(crange))
    
    for ic in eachindex(crange)
        c = crange[ic]
        ciclosestables!(xx, f, nit, nout, c)
        ff[:,ic] = xx
    end
    
    return ff
end



#Todo lo de AuDiff para que calcule la derivada 

type AuDiff{T<:Real} <: Real
    fun :: T
    der :: T
end
AuDiff(a, b) = AuDiff(promote(a,b)...)
AuDiff{T<:Real}(a::T) = AuDiff(a, zero(T))
import Base: convert

convert(::Type{AuDiff}, b::Real) = AuDiff(b)
convert{T<:Real, S<:Real}(::Type{AuDiff{T}}, b::S) = AuDiff(convert(T,b))
convert{T<:Real}(::Type{AuDiff{T}}, b::T) = AuDiff(b)
convert{T<:Real, S<:Real}(::Type{AuDiff{T}}, b::AuDiff{S}) = 
    AuDiff(convert(T,b.fun),convert(T,b.der))
import Base: promote_rule, promote

promote_rule{T<:Real}(::Type{AuDiff{T}}, ::Type{T}) = AuDiff{T}
promote_rule{T<:Real, S<:Real}(::Type{AuDiff{T}}, ::Type{S}) = AuDiff{promote_type(T, S)}
import Base: +, -, *, /, ^

+(a::AuDiff, b::AuDiff) = AuDiff(a.fun+b.fun, a.der+b.der)

-(a::AuDiff, b::AuDiff) = AuDiff(a.fun-b.fun, a.der-b.der)

*(a::AuDiff, b::AuDiff) = AuDiff(a.fun*b.fun, a.fun*b.der+a.der*b.fun)

function /(a::AuDiff, b::AuDiff)
    nn = a.fun/b.fun
    dd = (a.der - nn*b.der)/b.fun
    return AuDiff(nn, dd)
end

import Base: exp, log, sin, cos, tan

tup_fn   = [:exp, :log, :sin, :cos, :tan]
tup_fder = [:exp, :(x -> 1/x), :(x -> cos(x)), :(x -> -sin(x)), :(x -> (sec(x))^2)]

for (fn, fnder) in zip(tup_fn, tup_fder)
    @eval ($fn)(a::AuDiff) = AuDiff( ($fn)(a.fun), a.der*($fnder)(a.fun) )
end

#Tu función que crea polinomios

Qc(x,c) = x^2 + c
nombre(n::Int, c) = Symbol( string("Qc", n, c) )
function Qcs(n::Int, c, z)
    # Checo que `n` sea >= 1
    @assert n ≥ 1   # ≥ se obtiene con \ge<TAB>
    ex = :(x -> x^2 + $c)
    for i = 2:n
        ex = :(x -> $(ex)(x^2 + $c))
    end
    ex_ret = :($(nombre(n, z))(x) = $ex(x))
    return eval(ex_ret)
end

Qcs (generic function with 1 method)

Ok, aquí va la función

In [49]:
"""
puntosbifurc(f, nit, nout, crange,p,tol)

Itera el mapeo `f(x,c)` `nit+nout` veces en un rango de c en `crange`. 
Calcula `f(x,c)` de orden `p` en cada c del rango. Obtiene el valor de la derivada de cada  `f(x,c)`
para las últimas `nout` iteraciones. 

Cuando la derivada es tan cercana a -1 como lo permite `tol`, guarda la c presente y la cercanía.
"""
function puntosbifurc(f, nit::Int, nout::Int, crange,p::Int,tol = 0.1) #combiné lo que tenía Benet con lo tuyo.
    xx = Vector{Float64}(nout)
    ff = typeof(zeros(Float64, 4))[]
    fs = Function[]
    count = 1
    for ic in eachindex(crange)
        c = crange[ic]
        ciclosestables!(xx, f, nit, nout, c) #para cada c en el rango saca las últimas 'nout' iteraciones 
                                            # y las coloca en xx
        
        expre1 = Qcs(p, c, count)
        push!(fs, eval(expre1)) #lo mismo que tenías, hace el polinomio de grado p para cada c y las guarda en fs
        
        raices = Float64[]
        
        for xi in xx
            root = newton1D(x -> (fs[end](x) - x), xi)
            #@show root
            push!(raices, root)
        end
        for i in 1:nout
            near  = abs(1 - abs(fs[end](xdual(raices[i])).der))
            #el último polinomio de fs corresponde al de c presente. Al operar sobre el AuDiff de Benet para cada xx[i]
            #saca el valor de la función en los puntos de convergencia y cuánto vale la derivada ahí. 
            #por cierto, sí es -1, entonces le +1, saco abs y comparo con la toleracia
            if near < tol
                push!(ff,Float64[c,near,raices[i], fs[end](xdual(raices[i])).der]) #si < que la toleracia,guarda en ff la c con que se trabajó y qué tan cercano fue
            end
        end
        count += 1 
    end
    
    return ff
end

puntosbifurc

In [50]:
fzero(x -> x^2, 1)

2.3675455903731173e-8

In [52]:
crange = -0.50:-1/2^8:-1
xx = puntosbifurc(Qc, 1000, 5, crange, 1, 1e-15)

3-element Array{Array{Float64,1},1}:
 [-0.75,8.88178e-16,-0.5,-1.0]
 [-0.75,6.66134e-16,-0.5,-1.0]
 [-0.75,6.66134e-16,-0.5,-1.0]

In [53]:
##### c2
crange = -1:-1/2^8:-1.5
xx = puntosbifurc(Qc, 1000, 3, crange,2,1e-10)

3-element Array{Array{Float64,1},1}:
 [-1.25,1.90958e-14,-1.20711,-1.0]
 [-1.25,5.77316e-15,0.207107,-1.0]
 [-1.25,2.95319e-14,-1.20711,-1.0]

In [ ]:
-1.25 - 1/8

In [18]:
Qcs(8, -1.75, 13)

Qc813 (generic function with 1 method)

In [19]:
Qc813(1.30194)

-1.7469802516464983

In [54]:
#c3
crange = -1.25:-1/2^9:-1.5
xx = puntosbifurc(Qc, 1000, 10, crange,3,1e-2)

0-element Array{Array{Float64,1},1}

In [ ]:
Qc(0.184653,-1.21875)

In [ ]:
Qc(ans,-1.21875 )

In [16]:
#c4
crange = -1.875:-1/2^8:-2
xx = puntosbifurc(Qc, 1000, 10, crange,4,1e-3)

0-element Array{Array{Float64,1},1}

In [ ]:
#c5
crange = -1.36:-1/2^16:-1.405
xx = puntosbifurc(Qc, 2000, 50, crange,4,0.0001)

In [ ]:
#c5
crange = -1.36:-1/2^15:-1.405
xx = puntosbifurc(Qc, 1000, 50, crange,6,0.0001)

In [ ]:
a = 4.67
a_2 = 0.25
a_1 = -0.75
(1+1/a)*a_1 - (1/a)*a_2

**Ejercicio 2:**

Repitan el ejercicio anterior para el mapeo $S_c(x) = c \sin(x)$. ¿Cómo se comparan los valores obtenidos de $f_n$?

**Ejercicio 3:**

Como se ve en la Fig. 1 (de [este](https://github.com/lbenet/2017-1_TSFisComp/blob/master/notas_clase/08_Mapeos1d-3.ipynb) notebook), $x=0$ pertenece a un ciclo de periodo $2^n$ para ciertos valores $C_n$ del parámetro. Dichos valores son *especiales*, ya que $x=0$ esté en el ciclo de periodo $2^n$ marca los llamados *ciclos superestable*, donde tenemos $(Q^{2^p}_{C_n})'(0)=0$.

¿A qué converge la secuencia $f_n$, definida ahora con los valores $C_n$.

De los $2^p$ puntos del ciclo de periodo $2^p$, es decir, $\{0, p_1, \dots p_{2^{n-1}}\,\}$ hay uno (distinto del 0) cuya distancia a 0 es menor; a ese punto lo identificamos como $d_n$. Calcular numéricamente a dónde converge la secuencia $d_n/d_{n+1}$.

In [ ]:
func = Function[x+1,x+3]